In [1]:
library(rdydisstools)
setwd('~/notebooks/dissertation')
loadpackages(c('doParallel', 'foreach', 'logging', 'glue', 'grid', 'gridExtra', 'dplyr', 'ggplot2'))
nDatasets <- 5

In [2]:
registerDoParallel(72)
compmethods <- foreach(i=1:nDatasets, .verbose=FALSE) %dopar% {
    # Initialize logging
    addHandler(writeToFile, file=glue::glue('~/notebooks/dissertation/logs/rq2/sim{i}.log'), level='INFO')
    logstring <- glue::glue('dis.rq2.{i}')
    
    # Specify path for sample
    datasetstring <- glue::glue("~/notebooks/dissertation/data/simsample{i}.RData")
    logging::loginfo(glue::glue('dataset loaded from data/simsample{i}'), logger=logstring)
    
    # load sample dataset into memory
    load(datasetstring)
    logging::loginfo(glue::glue('dataframe has been loaded from: {datasetstring}'), logger=logstring)
    
    # Evaluate RQ 2
    dispatchRQ2(df, lastColumn=100, LS_CUT=.4, EO_CUT=.2, OUT_CUT=.5, i=i)
} %>% do.call("rbind.data.frame", .)

save(compmethods, file='~/notebooks/dissertation/artifacts/rq2/compmethods.RData')

truth,ls,eo,out,outsq
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,4,0.6787363,128.31077,2.04723685
0,3,0.9546542,104.09685,0.30853832
0,4,0.6727644,98.84040,0.06890490
0,5,0.9817171,98.34184,0.10470420
0,4,0.8508711,105.25341,0.39158592
0,3,0.8036790,106.09886,0.45229454
0,4,0.6512379,104.27834,0.32157033
0,6,0.4368415,93.00670,0.48779766
0,4,0.8860718,111.78616,0.86067456


In [26]:
save(ls, file=glue::glue("~/notebooks/dissertation/artifacts/rq2/scores/sim1.RData"))

In [ ]:
ggplot(maxJ, aes(ls)) + geom_histogram() + stat_bin(breaks=seq(-3,3,.1), binwidth=.1) + scale_x_continuous(breaks=seq(-3,3,1))
ggplot(maxJ, aes(eo)) + geom_histogram() + stat_bin(breaks=seq(-3,3,.1), binwidth=.1) + scale_x_continuous(breaks=seq(-3,3,1))
ggplot(maxJ, aes(md)) + geom_histogram() + stat_bin(breaks=seq(-3,3,.1), binwidth=.1) + scale_x_continuous(breaks=seq(-3,3,1))

In [2]:
clearExperiment('rq2')

Warning message in file.remove(glue::glue("logs/{hypothesis}/*")):
“cannot remove file 'logs/rq1/*', reason 'No such file or directory'”Warning message in file.remove(glue::glue("artifacts/{hypothesis}/ls/*")):
“cannot remove file 'artifacts/rq1/ls/*', reason 'No such file or directory'”Warning message in file.remove(glue::glue("artifacts/{hypothesis}/eo/*")):
“cannot remove file 'artifacts/rq1/eo/*', reason 'No such file or directory'”Warning message in file.remove(glue::glue("artifacts/{hypothesis}/md/*")):
“cannot remove file 'artifacts/rq1/md/*', reason 'No such file or directory'”

[1] FALSE

In [3]:
file.remove('~/notebooks/dissertation/logs/rq2/*')

Warning message in file.remove("~/notebooks/dissertation/logs/rq1/*"):
“cannot remove file '~/notebooks/dissertation/logs/rq1/*', reason 'No such file or directory'”

[1] FALSE

In [4]:
devtools::install_github("ryentes/rdy-dissertation-tools")


✔  checking for file ‘/tmp/Rtmp2WhboA/remotes725249e5b365/ryentes-rdy-dissertation-tools-a94914e/DESCRIPTION’
─  preparing ‘rdydisstools’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  looking to see if a ‘data/datalist’ file should be added
─  building ‘rdydisstools_0.0.2.tar.gz’
   


Installing package into ‘/usr/lib64/R/library’
(as ‘lib’ is unspecified)
